In [94]:
import re
import torch
import random
import evaluate
import Levenshtein
import pandas as pd 
import numpy as np

from datasets import Dataset
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, EarlyStoppingCallback, BertTokenizerFast

### Data 

In [2]:
df = pd.read_excel("data/data.xlsx")  

texts = df["text"].astype(str).tolist()
entities = df["entity"].astype(str).tolist()

data = list(zip(texts, entities))
random.shuffle(data)

train_size = int(0.8 * len(data))  

train_data = data[:train_size]
test_data = data[train_size:]

In [5]:
train_data[0] , test_data[0]

(('Набор губцевого инструмента ЗУБР "Профессионал", до 1000 В, 12 предметов, арт. 2214-H12_z01',
  '2214-H12_z01'),
 ('Выключатель автоматический МS116 6,3-10 A, 50 kA, 1SAM250000R1010 ABB',
  '1SAM250000R1010'))

In [7]:
train_data.__len__(), test_data.__len__()

(8192, 2048)

### Tokes-model

In [8]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

### Prepare Dataset

In [ ]:
def preprocess_example(example, tokenizer):
    text = example["text"]
    labels = example["labels"]  

    tokenized_inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt",
        return_offsets_mapping=True
    )

    aligned_labels = []
    offset_mapping = tokenized_inputs.offset_mapping[0] 

    for start, end in offset_mapping:
        if start == end:
            aligned_labels.append(-100)  
        else:
            aligned_labels.append(labels[start])

    tokenized_inputs["labels"] = aligned_labels
    return {
        "input_ids": tokenized_inputs["input_ids"][0],
        "attention_mask": tokenized_inputs["attention_mask"][0],
        "labels": torch.tensor(aligned_labels), 
    }

def create_char_labeled_dataset(data):
    examples = []
    for text, entity in data:

        labels = [0] * len(text)
        start_idx = text.find(entity)

        if start_idx != -1:
            for i in range(start_idx, start_idx + len(entity)):
                labels[i] = 1

        examples.append({
            "text": text,
            "labels": labels,
            "entity": entity
        })

    return Dataset.from_list(examples)

In [10]:
dataset_train = create_char_labeled_dataset(train_data)
dataset_test = create_char_labeled_dataset(test_data)

In [16]:
print(f"Test-dataset - {dataset_test}\nTrain-dataset - {dataset_train}")

Test-dataset - Dataset({
    features: ['text', 'labels', 'entity'],
    num_rows: 2048
})
Train-dataset - Dataset({
    features: ['text', 'labels', 'entity'],
    num_rows: 8192
})


In [20]:
print(dataset_test[0])
print(dataset_train[0])

{'text': 'Выключатель автоматический МS116 6,3-10 A, 50 kA, 1SAM250000R1010 ABB', 'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], 'entity': '1SAM250000R1010'}
{'text': 'Набор губцевого инструмента ЗУБР "Профессионал", до 1000 В, 12 предметов, арт. 2214-H12_z01', 'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'entity': '2214-H12_z01'}


In [42]:
tokens_train = dataset_train.map(lambda x: preprocess_example(x, tokenizer)).remove_columns(["text", "entity"])
tokens_test = dataset_test.map(lambda x: preprocess_example(x, tokenizer)).remove_columns(["text", "entity"])

tokens_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokens_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map: 100%|██████████| 2048/2048 [00:01<00:00, 1602.08 examples/s]


In [43]:
print(f"Test-dataset - {tokens_test}\nTrain-dataset - {tokens_train}")

Test-dataset - Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2048
})
Train-dataset - Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 8192
})


### Model

In [39]:
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=2, 
    ignore_mismatched_sizes=True
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="ner_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    report_to="none", 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokens_train,
    eval_dataset=tokens_test,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

### Training 

In [50]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.015200,0.029360
2,0.014300,0.022094
3,0.007500,0.023182
4,0.005000,0.023831
5,0.003400,0.026105


TrainOutput(global_step=2560, training_loss=0.008893972673104145, metrics={'train_runtime': 762.7568, 'train_samples_per_second': 107.4, 'train_steps_per_second': 6.712, 'total_flos': 2675678788976640.0, 'train_loss': 0.008893972673104145, 'epoch': 5.0})

### EVAL

In [ ]:
def predict_entity(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        logits = model(**inputs).logits
    predictions = torch.argmax(logits, dim=2).squeeze().tolist()

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze())
    pred_labels = predictions

    filtered_tokens = [
        token.replace("#", "") 
        for token, label in zip(tokens, pred_labels)
        if label == 1 and token not in tokenizer.all_special_tokens
    ]

    predicted_entity = "".join(filtered_tokens)  
    return predicted_entity

def char_accuracy(gt, pred):
    gt = str(gt).replace(" ", "")
    pred = str(pred).replace(" ", "")

    correct = sum(1 for a, b in zip(gt, pred) if a == b)
    max_len = max(len(gt), len(pred))
    return correct / max_len if max_len > 0 else 0

def avg_levenshtein(true_list, pred_list):
    distances = [Levenshtein.distance(gt, pred) for gt, pred in zip(true_list, pred_list)]
    return sum(distances) / len(distances)

def sequence_level_accuracy(true_list, pred_list):
    correct = sum(1 for gt, pred in zip(true_list, pred_list) 
                  if gt.replace(" ", "") == pred.replace(" ", ""))
    return correct / len(true_list)

In [58]:
true_entities = [example["entity"] for example in dataset_test]
texts = [example["text"] for example in dataset_test]

In [63]:
for ent, text in zip(true_entities, texts):
    print(f"|TEXT| - {text} : |ENT| - {ent}")

|TEXT| - Выключатель автоматический МS116 6,3-10 A, 50 kA, 1SAM250000R1010 ABB : |ENT| - 1SAM250000R1010
|TEXT| - Направляющая центральная закрытия клапанов S320916284A Siat : |ENT| - S320916284A
|TEXT| - Камера тормозная Alon VL0301078 : |ENT| - VL0301078
|TEXT| - Нож 3ʺ( № детали по каталогу PW-557 F: C-50311-01) : |ENT| - C-50311-01
|TEXT| - ПОРШЕНЬ R256272 : |ENT| - R256272
|TEXT| - Комплект уплотнений № 55025293 : |ENT| - 55025293
|TEXT| - РЕМКМП УПЛОТНЕНИЙ ФОРСУНКИ Д-260 2135 : |ENT| - 2135
|TEXT| - САЙЛЕНТБЛОК ПОДРАМНИКА 1695011 : |ENT| - 1695011
|TEXT| - Турбокомпрессор CV18504 : |ENT| - CV18504
|TEXT| - ШТАНГА 50X25X1868 H2K 405 1504 : |ENT| - 405 1504
|TEXT| - SVK-МУФТА M22*1,5 375 2737 : |ENT| - 375 2737
|TEXT| - Набор QIAGEN Large-Construct Kit для очистки больших ДНК-конструкций от примесей геномной ДНК Артикул:Q-12462 : |ENT| - 12462
|TEXT| - Набор отверток в ложементе шлиц/PH/PZ, 8 предметов, тип N151-161-008, NORGAU 062204008 : |ENT| - 062204008
|TEXT| - Вал 4092.100501

In [64]:
predicted_entities = [predict_entity(text, model, tokenizer) for text in texts]

In [68]:
for i in predicted_entities:
    print(f"|PREDICTIONS| - {i}")

|PREDICTIONS| - 1SAM250000R1010
|PREDICTIONS| - S320916284A
|PREDICTIONS| - VL0301078
|PREDICTIONS| - PW-557F:C-50311-01
|PREDICTIONS| - R256272
|PREDICTIONS| - 55025293
|PREDICTIONS| - 2135
|PREDICTIONS| - 1695011
|PREDICTIONS| - CV18504
|PREDICTIONS| - 4051504
|PREDICTIONS| - 3752737
|PREDICTIONS| - 12462
|PREDICTIONS| - 062204008
|PREDICTIONS| - 4092.1005010
|PREDICTIONS| - 3163-00-3801010-051
|PREDICTIONS| - 3568678
|PREDICTIONS| - 82357050100
|PREDICTIONS| - 5350-3518001-01
|PREDICTIONS| - 3823024
|PREDICTIONS| - 48530-80701
|PREDICTIONS| - SN350992
|PREDICTIONS| - 100055871
|PREDICTIONS| - 4381503
|PREDICTIONS| - ЦНС60-165-1.01.031
|PREDICTIONS| - 60126222
|PREDICTIONS| - 853W013
|PREDICTIONS| - 3475597
|PREDICTIONS| - 5533400200
|PREDICTIONS| - 250854R-B
|PREDICTIONS| - 700-40-3141
|PREDICTIONS| - 240-1029228-А
|PREDICTIONS| - 10031
|PREDICTIONS| - 130-3705010-В
|PREDICTIONS| - 4825861
|PREDICTIONS| - CF500
|PREDICTIONS| - 46193680
|PREDICTIONS| - 4902
|PREDICTIONS| - 111-40-010

In [69]:
accuracies = [char_accuracy(gt, pred) for gt, pred in zip(true_entities, predicted_entities)]
avg_accuracy = sum(accuracies) / len(accuracies)

print(f"Средняя символьная точность: {avg_accuracy:.4f}")

Средняя символьная точность: 0.9780


In [76]:
print(f"Средняя дистанция Levenshtein: {avg_levenshtein(true_entities, predicted_entities):.4f}")

Средняя дистанция Levenshtein: 0.2002


In [82]:
print(f"Точность совпадения строк: {sequence_level_accuracy(true_entities, predicted_entities):.4f}")

Точность совпадения строк: 0.9683


### Report 

#### 📊 Отчет по оценке качества модели 

В рамках задачи дообучения модели BERT на посимвольную классификацию была проведена оценка качества предсказаний модели по нескольким ключевым метрикам. Ниже приведены итоговые результаты. 

| Метрики                        |       Значение      |
|--------------------------------|---------------------|
| Средняя символьная точность    |        0.9780       |
| Средняя дистанция Левенштейна  |        0.2002       |
| Точность совпадения строк      |        0.9683       |


#### Описание метрик

- ✅    Средняя символьная точность: 0.9780 

Для каждой строки рассчитывалась доля правильно восстановленных символов относительно максимальной длины эталона и предсказания. 
```Модель в среднем корректно восстанавливает 97.8% символов , что говорит о высокой точности на уровне отдельных знаков.```

- ✅    Средняя дистанция Левенштейна: 0.2002 

Рассчитано среднее количество операций (вставка, удаление, замена), необходимых для превращения предсказанной строки в эталонную. 
```Среднее значение близко к нулю, что говорит о том, что ошибки редкие и локальные. В большинстве случаев модель восстанавливает строки с минимальными или без изменений .```

- ✅    Точность совпадения строк: 0.9683 

Доля строк, которые были восстановлены полностью точно , без каких-либо отличий от эталона. 
```96.8% примеров  были восстановлены идеально — это очень высокий показатель полного совпадения выходной строки с ожидаемой.```


#### ✅ Окончательный вердикт
Модель демонстрирует очень высокое качество предсказаний : 

    - Высокая точность на уровне символов.
    - Почти полное восстановление строк целиком.
    - Минимальное количество ошибок и их серьёзности.
     

Таким образом, модель готова к использованию в production-среде с возможностью дальнейшей тонкой настройки под конкретные типы ошибок. 